In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from tqdm import tqdm

In [ ]:
with open('data.pkl', 'rb') as f:
    review = pickle.load(f)
item_meta = np.load('item.npy').item()

In [ ]:
def IntegratedData(item_data, review_data, threshold = 15):
    data = dict()
    for key in list(review_data.keys()):
        item_list = []
        for item in list(review_data[key]):
            try:
                id_ = item['asin']
                img, price, category, brand = item_data[id_]
                item['img'], item['price'], item['category'], item['brand'] = img, price, category, brand
                item_list.append(item)
            except:
                pass
        if len(item_list) >= threshold:
            data[key] = item_list
    return data

In [ ]:
data = IntegratedData(item_meta, review, 5)

In [ ]:
with open('merge_data.pkl', 'wb') as f:
    pickle.dump(data, f)

# For count

In [ ]:
count = dict()
for key in list(data.keys()):
    length = len(data[key])
    if length in count.keys():
        count[length] = count[length] +1
    else:
        count[length] = 1

In [ ]:
count

In [ ]:
num = 0
for i in count.keys():
    if i>=15:
        num += count[i]

In [ ]:
num

# split train / val / test

In [2]:
with open('merge_data.pkl', 'rb') as f:
    data = pickle.load(f)

In [3]:
len(list(data.keys()))

61295

In [4]:
def one_hot_dict(data_, category):
    series = set()
    for key_ in list(data_.keys()):
        for item_ in data_[key_]:
            if category == 'category':
                for cate in item_[category][0]:
                    series.add(cate)
            else:
                series.add(item_[category])

    result_dict = dict()
    for i, cell in enumerate(series):
        one_hot = np.zeros(len(series), dtype = np.float32)
        one_hot[i] = 1
        result_dict[cell] = one_hot
    
    return result_dict

In [5]:
asin_dict = one_hot_dict(data, 'asin')
brand_dict = one_hot_dict(data, 'brand')
category_dict = one_hot_dict(data, 'category')

In [9]:
len(brand_dict.keys()), len(category_dict.keys())

(9033, 1007)

brand, cate, img, price, review

In [22]:
def split_data(data_, brand_dict, asin_dict, category_dict, mode,
               n = 4, k = 2, val_num = 2, test_num = 2):
    if n <= k :
        print("invalid argument")
        return -1
    result_imgs, result_reviews, result_meta, result_asins = [], [], [], []
    brand_len , asin_len, category_len = len(brand_dict.keys()), len(asin_dict.keys()), len(category_dict.keys())
    
    for key_ in list(data_.keys()):
        length = len(data_[key_])
        # this is temporary threshold. it need to modify after discussing
        # ex, if n = 5, k = 2, val = 3, test = 4, then length = 17, (0,10) (10,13) (13,17) 
        if length >= n + val_num + test_num + 1:
            num_instance = length - n + 1 
            if mode == 'train':
                start, end = 0, num_instance - (val_num + test_num)
            elif mode == 'val':
                start, end = num_instance - (val_num + test_num) , num_instance - test_num
            else:
                start, end = num_instance - test_num, num_instance 
            
            for num in range(start, end):
                tmp_imgs, tmp_reviews, tmp_meta, tmp_asin = [], [], [], []
                for count in range(0,n-k):
                    tmp_item = data_[key_][num + count]
                    tmp_imgs.append(tmp_item['img'])
                    tmp_reviews.append(tmp_item['reviewText'])
                    tmp_categories = np.zeros(category_len)
                    for tmp_category in tmp_item['category'][0]:
                        tmp_categories += category_dict[tmp_category]
                    tmp_brand = brand_dict[tmp_item['brand']]
                    tmp_price = np.array([tmp_item['price']])
                    tmp_meta.append(np.concatenate([tmp_categories, tmp_brand, tmp_price]))
                for count in range(n-k,n):
                    tmp_item = data_[key_][num + count]
                    tmp_asin.append(asin_dict[tmp_item['asin']])
                result_imgs.append(np.array(tmp_imgs))
                result_reviews.append(np.array(tmp_reviews))
                result_meta.append(np.array(tmp_meta))
                result_asins.append(np.array(tmp_asin))
    return (result_imgs, result_reviews, result_meta, result_asins) 

In [ ]:
dataset = split_data(data_ = data, brand_dict = brand_dict , asin_dict = asin_dict,
           category_dict = category_dict, mode = 'train')